In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('project_final.csv')

/Users/nalliennallien/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (23,35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df = df.drop(['Unnamed: 0.1.1','Unnamed: 0','Unnamed: 0.1','EVENT_ID_CNTY','EVENT_DATE','ASSOC_ACTOR_2','ASSOC_ACTOR_1' ,'TIME_PRECISION', 'INTER1', 'INTER2', 'INTERACTION', 'ISO', 'REGION', 'ADMIN1' , 'ADMIN1', 'ADMIN2', 'ADMIN3', 'LOCATION', 'LATITUDE', 'LONGITUDE','GEO_PRECISION', 'SOURCE', 'SOURCE_SCALE', 'TAGS', 'TIMESTAMP'], axis=1)



### Вот наша маленькая табличка, но в ней куча нанов, из-за того, что спаршенные таблички не полностью стыкуются с нашем исходником

In [4]:
df.head() 

,YEAR,DISORDER_TYPE,EVENT_TYPE,SUB_EVENT_TYPE,ACTOR1,ACTOR2,CIVILIAN_TARGETING,COUNTRY,NOTES,FATALITIES,index,classification
0,1997,Political violence,Violence against civilians,Attack,GIA: Armed Islamic Group,Civilians (Algeria),Civilian targeting,Algeria,5 January: Beheading of 5 citizens in Douaouda...,5,0.624,Lower middle income
1,1997,Political violence,Violence against civilians,Attack,GIA: Armed Islamic Group,Civilians (Algeria),Civilian targeting,Algeria,Two citizens were beheaded in Hassasna.,2,0.624,Lower middle income
2,1997,Political violence,Violence against civilians,Attack,GIA: Armed Islamic Group,Civilians (Algeria),Civilian targeting,Algeria,Two citizens were killed in a raid on the vill...,2,0.624,Lower middle income
3,1997,Political violence,Violence against civilians,Attack,GIA: Armed Islamic Group,Civilians (Algeria),Civilian targeting,Algeria,4 January: 16 citizens were murdered in the vi...,16,0.624,Lower middle income
4,1997,Political violence,Violence against civilians,Attack,GIA: Armed Islamic Group,Civilians (Algeria),Civilian targeting,Algeria,5 January: Killing of 18 citizens in the Olivi...,18,0.624,Lower middle income


In [5]:
df.isna().sum()

YEAR                        0
DISORDER_TYPE               0
EVENT_TYPE                  0
SUB_EVENT_TYPE              0
ACTOR1                      0
ACTOR2                 880666
CIVILIAN_TARGETING    1475228
COUNTRY                     0
NOTES                       0
FATALITIES                  0
index                  512784
classification         156431
dtype: int64

##### Сейчас нам нужно затереть немного нанов в index (512784) и в classification (156431). Будем делать это функцией, а не как в прошлый раз

In [14]:
# тут я ищу страны в которых мало наблюдений, их сразу удаляем, а остальные будем заполнять информацией из внешних источников
nuli_all_del, nuli_class, nuli_ind = [], [], []
for el in df['COUNTRY'].unique():
    if sum(df[df['COUNTRY'] == el]['classification'].isnull()):
        nuli_class.append(el)
    if sum(df[df['COUNTRY'] == el]['index'].isnull()):
        nuli_ind.append(el)

ll_del_class = []
ll_find_class = []
for i in nuli_class:
    n_n = df[df['COUNTRY'] == i].shape[0]
    if n_n < 550:
        ll_del_class.append(i)
        nuli_all_del.append(i)
    else:
        ll_find_class.append(i)


ll_del_ind = []
ll_find_ind = []
for i in nuli_ind:
    n_n = df[df['COUNTRY'] == i].shape[0]
    if n_n < 550:
        ll_del_ind.append(i)
        nuli_all_del.append(i)
    else:
        ll_find_ind.append(i)


In [8]:
def chistka(method, country, year, data, data_class, data_ind):
    
    if method == 'del':
        ind = data[(data['COUNTRY'] == country)].index
        data = data.drop(ind)
                   
    if method == 'class':
        for i in range(len(year)): 
            ind = data[(data['COUNTRY'] == country) & (data['YEAR'] == year[i])].index
            data.loc[ind, 'classification'] = data_class[i]  
                   
    if method == 'ind':
        for i in range(len(year)): 
            ind = data[(data['COUNTRY'] == country) & (data['YEAR'] == year[i])].index
            data.loc[ind, 'index'] = data_ind[i]
            
    if method == 'class_ind':
        for i in range(len(year)): 
            ind = data[(data['COUNTRY'] == country) & (data['YEAR'] == year[i])].index
            data.loc[ind, 'index'] = data_ind[i]
            data.loc[ind, 'classification'] = data_class[i]
                   
    return data
            

In [9]:
# удалим все страны где мало наблюдений и много пропусков 
countries = nuli_all_del
for i in countries:
    df = chistka(method = 'del', 
        country = i, 
        year = 0, 
        data = df, 
        data_class = 0, 
        data_ind = 0)

In [10]:
# починим все страны, где надо добавлять и индекс и класс
countries_par = ['Venezuela', 'United States', 'Taiwan', 'Bermuda', 'China', 'Democratic Republic of Congo', 'Ivory Coast', 'North Macedonia', 'Turkey']
years = [np.arange(2018, 2024), 
         np.arange(2020, 2024),
         np.arange(2018, 2024),
         np.arange(2021, 2024),
         np.arange(2018, 2024),
         np.arange(1997, 2024),
         np.arange(1997, 2024),
         np.arange(2018, 2024),
         np.arange(2016, 2024)
        ]
indexes = [[0.74, 0.72, 0.69, 0.69, 0.65, 0.64],
           [0.92, 0.93, 0.93, 0.93, 0.93],
           [0.907 for j in np.arange(2018, 2024)],
           [0.981 for i in np.arange(2021, 2024)],
           [0.755, 0.755, 0.755, 0.751, 0.751, 0.751],
           [0.37, 0.37, 0.38, 0.37, 0.38, 0.37, 0.38, 0.38, 0.39, 0.40, 0.40, 0.41, 0.42, 0.42, 0.43, 0.44, 0.44, 0.45, 0.46, 0.46, 0.47, 0.47, 0.48, 0.48, 0.48, 0.48, 0.48],
           [0.46, 0.46, 0.46, 0.46, 0.46, 0.45, 0.45, 0.46, 0.46, 0.46, 0.46, 0.46, 0.47, 0.47, 0.47, 0.47, 0.48, 0.48, 0.50, 0.51, 0.52, 0.53, 0.54, 0.55, 0.55, 0.55, 0.55],
           [0.779, 0.779, 0.779,0.779, 0.779, 0.7],
           [0.82, 0.83, 0.84, 0.84, 0.83, 0.84, 0.84, 0.84]
          ]
classes = [['Upper middle income', 'Upper middle income', 'Lower middle income', 'Lower middle income', 'Lower middle income', 'Lower middle income'],
           ['High income' for i in np.arange(2020, 2024)],
           ['High income' for i in np.arange(2018, 2024)],
           ['High income' for i in np.arange(2018, 2024)],
           ['Upper middle income' for i in np.arange(2018, 2024)],
           ['Low income' for i in np.arange(1997, 2024)],
           ['Low income' for j in range(11)] + ['Lower middle income' for j in range(16)],
           ['Upper middle income' for i in np.arange(2018, 2024)],
           ['Upper middle income' for i in np.arange(2016, 2024)]
          ]
for i in range(len(countries_par)):
    chistka(method = 'class_ind', country = countries_par[i], year = years[i], data = df, data_class = classes[i], data_ind = indexes[i])



In [12]:
# починим все страны, где надо добавлять только индекс
countries_par = ['Angola', 'Chad', 'Ethiopia', 'Nigeria', 'Guinea-Bissau', 'Palestine', 'Czech Republic', 'Vietnam', 'Somalia', 'Kosovo', 'Puerto Rico', 'Burkina Faso', 'Madagascar', 'Liberia', 'eSwatini']
years  = [np.arange(1997, 2012),
          np.arange(1997, 2012),
          np.arange(1997, 2024),
          np.arange(1997, 2003),
          np.arange(1997, 2005),
          np.arange(1997, 2024),
          np.arange(2020, 2024),
          np.arange(2010, 2024),
          np.arange(1997, 2024),
          np.arange(2018, 2024),
          np.arange(2018, 2024),
          np.arange(1997, 1999),
          np.arange(1997, 1999),
          np.arange(1997, 1999),
          np.arange(1997, 2024)
         ]
indexes = [[0, 0.353, 0.356, 0.354, 0.364, 0.375, 0.386, 0.403, 0.420, 0.433, 0.447, 0.459, 0.475, 0.486, 0.500, 0.510, 0.526, 0.541, 0.552, 0.563, 0.582, 0.596, 0.597, 0.595, 0.595, 0.590,0.586],
           [0.288, 0.288, 0.288, 0.291, 0.301, 0.307, 0.307, 0.325, 0.327, 0.327, 0.337, 0.341, 0.355, 0.362, 0.372],
           [0, 0, 0, 0.287, 0.301,0.309,0.313,0.327,0.346,0.363,0.380,0.395,0.402,0.412,0.422,0.430,0.441,0.450,0.460,0.470,0.480,0.489,0.498,0.498,0.498, 0.499, 0.498],
           [0.393, 0.393, 0.455, 0.455,0.455, 0.455],
           [0.291, 0.299, 0.311, 0.512,0.522,0.534,0.537, 0.545],
           [0.650, 0.613, 0.623, 0.643, 0.645, 0.648, 0.652, 0.651, 0.662, 0.663, 0.672, 0.677, 0.682, 0.687, 0.694, 0.701, 0.703, 0.698, 0.710, 0.715, 0.719, 0.723, 0.727, 0.716, 0.715, 0.714,0.711],
           [0.892, 0.889, 0.890, 0.891],
           [0.663,0.668,0.672,0.676,0.680,0.684,0.688,0.692,0.697,0.703,0.710,0.703, 0.701, 0.695],
           [0, 0.1, 0.1, 0.12, 0.2, 0.217, 0.226,0.230,0.232,0.240,0.253,0.285,0.315,0.341,0.360,0.378,0.396,0.399,0.405,0.409,0.415,0.420,0.428,0.436,0.444,0.444,0.444],
           [0.755, 0.752, 0.732, 0.732, 0.721, 0.735],
           [0.754, 0.757, 0.761, 0.760, 0.765, 0.766],
           [0.281, 0.285],
           [0.421, 0.431],
           [0.361, 0.383],
           [0.518, 0.509, 0.500, 0.490, 0.482,0.471,0.463,0.451,0.446,0.444,0.446,0.462,0.471,0.479,0.486,0.503,0.517,0.531,0.546,0.559,0.575, 0.586,0.596,0.607,0.615,0.610,0.597]
          ]

for i in range(len(countries_par)):
    chistka(method = 'ind', country = countries_par[i], year = years[i], data = df, data_class = 0, data_ind = indexes[i])


In [13]:
# так же во время присоединения файлов некоторые индексы съехали, тут мы их поправляем
for can in ll_find_ind:
    value = df[(df['COUNTRY'] == can) & (df['YEAR'] == 2021)]['index'].iloc[0]
    ind_22 = df[(df['COUNTRY'] == can) & (df['YEAR'] == 2022)]['index'].index
    ind_23 = df[(df['COUNTRY'] == can) & (df['YEAR'] == 2023)]['index'].index
    df.loc[ind_22, 'index'] = value
    df.loc[ind_23, 'index'] = value

In [19]:
df.isna().sum()

YEAR                        0
DISORDER_TYPE               0
EVENT_TYPE                  0
SUB_EVENT_TYPE              0
ACTOR1                      0
ACTOR2                 872631
CIVILIAN_TARGETING    1464533
COUNTRY                     0
NOTES                       0
FATALITIES                  0
index                       0
classification              0
dtype: int64

In [20]:
df.to_csv('chist_df.csv', encoding='utf-8')

__В итоге мы починили наш датасет и тепрь у нас нет пропусков в целевых переменных. Далее можно заняться визуализациями и проверкой гипотез__